In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import sklearn.model_selection
from HW1.logit import Logit
from datetime import datetime

In [2]:
def read_dataset(path):
    data = pd.read_csv(path)
    X = data.iloc[:,:-1].values
    y = data.iloc[:, -1].apply(lambda c: 1 if c == 'P' else -1).values
    return X, y

In [3]:
def calc_f_score(X, y, alpha, solver, debug_iters):
    n_splits = 5
    cv = sklearn.model_selection.KFold(n_splits=n_splits, shuffle=True)
    mean_f_score = 0.0
    for train_indexes, test_indexes in cv.split(X):
        X_train = X[train_indexes]
        X_test = X[test_indexes]
        y_train = y[train_indexes]
        y_test = y[test_indexes]

        classifier = Logit(alpha, solver)
        classifier.fit(X_train, y_train, debug_iters)
        y_pred = classifier.predict(X_test)

        tp = np.sum((y_pred == 1) & (y_test == 1))
        fp = np.sum((y_pred == 1) & (y_test != 1))
        tn = np.sum((y_pred != 1) & (y_test != 1))
        fn = np.sum((y_pred != 1) & (y_test == 1))

        if tp != 0:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f_score = 2 * precision * recall / (precision + recall)
            mean_f_score += f_score
    return mean_f_score / n_splits

In [4]:
def get_best_param(X, y, solver, debug_iters):
    best_alpha = None
    max_f_score = -1
    for alpha in [0.0001, 0.001, 0.01, 0.1, 1.]:
        cur_f_score = calc_f_score(X, y, alpha, solver, debug_iters)
        print('alpha =', alpha, 'f-score =', cur_f_score)
        if cur_f_score > max_f_score:
            max_f_score = cur_f_score
            best_alpha = alpha
    return best_alpha, max_f_score

In [5]:
def draw(clf, X, ans, step_x, step_y):
    x_min, y_min = np.amin(X, axis = 0)
    x_max, y_max = np.amax(X, axis = 0)
    x_min -= step_x
    x_max += step_x
    y_min -= step_y
    y_max += step_y
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step_x), np.arange(y_min, y_max, step_y))
    
    zz = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    plt.figure(figsize=(12, 12))
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    x0, y0 = X[ans != 1].T
    x1, y1 = X[ans == 1].T

    plt.pcolormesh(xx, yy, zz, cmap=matplotlib.colors.ListedColormap(['#FFAAAA', '#AAAAFF']))
    plt.scatter(x0, y0, color='red', s=100)
    plt.scatter(x1, y1, color='blue', s=100)
    plt.xlabel('x')
    plt.ylabel('y')
    
    plt.show()

In [6]:
def process_with_solver(X, y, solver, step_x, step_y, debug_iters=None):
    best_alpha, max_f_score = get_best_param(X, y, solver, debug_iters)
    print('Best params:', best_alpha, max_f_score)
    best_classifier = Logit(best_alpha, solver)
    start_time = datetime.now()
    number_of_steps = best_classifier.fit(X, y, debug_iters=debug_iters)
    end_time = datetime.now()
    timedelta = end_time - start_time
    if solver == 'newton':
        print('errors =', number_of_steps.errors, 'steps =', number_of_steps.steps)
    else:
        print('steps =', number_of_steps.steps)
    print('time =', timedelta.microseconds, 'microseconds')
    draw(best_classifier, X, y, step_x, step_y)

In [7]:
X, y = read_dataset('data/geyser.csv')

In [ ]:
process_with_solver(X, y, 'gradient', 0.1, 0.01, debug_iters=1000)

iteration = 1000 grad = [-0.00464144 -0.00090692 -0.00097621] ||grad|| = 0.004828921551180213
iteration = 2000 grad = [-1.54446147e-04 -3.10531480e-05 -2.96792634e-05] ||grad|| = 0.00016030835603089065
iteration = 1000 grad = [ 0.00446707  0.00126516 -0.00037979] ||grad|| = 0.004658278984811304
iteration = 2000 grad = [ 1.55227126e-04  4.41017762e-05 -1.29780478e-05] ||grad|| = 0.00016189149817404325
iteration = 1000 grad = [-0.00735265 -0.00138452 -0.00176316] ||grad|| = 0.0076868162301979275
iteration = 2000 grad = [-4.71998635e-04 -9.09411015e-05 -1.08054152e-04] ||grad|| = 0.0004926750398847003
iteration = 1000 grad = [ 0.00655028  0.00181027 -0.00062579] ||grad|| = 0.006824583128068837
iteration = 2000 grad = [ 2.26398223e-04  6.17849756e-05 -1.69331955e-05] ||grad|| = 0.00023528763609532866
iteration = 1000 grad = [-0.02880901  0.00119139 -0.03245927] ||grad|| = 0.04341639091110138
iteration = 2000 grad = [-0.01558297  0.00059598 -0.01756981] ||grad|| = 0.02349217708886017
iterat

In [ ]:
process_with_solver(X, y, 'newton', 0.1, 0.01)